# Logo Detection - YOLOv7 and YOLOv11
This notebook provides options for training both YOLOv7 and the newer YOLOv11 models for logo detection.

# Clone the repo
Get the repo and submodules

In [ ]:
!git clone https://github.com/nuwandda/yolov7-logo.git

Download submodules.

In [ ]:
%cd yolov7-logo/
!git submodule update --init

Install the necessary packages.

In [ ]:
!pip install -r src/requirements.txt

# Choose Your Model
Select which YOLO version you want to train:

In [ ]:
# Set to 'yolov7' or 'yolo11'
YOLO_VERSION = 'yolo11'

# Download the dataset
Run the **getFlickr.sh** file to download the dataset.

In [ ]:
!sh data/getFlickr.sh

# Prepare data
We need to convert data to YOLO format. Now that we have our dataset, we need to convert the annotations into the format expected by YOLO. YOLO expects data to be organized in a specific way, otherwise it is unable to parse through the directories.

In [ ]:
!python src/convert_annotations.py --dataset flickr27

To see if the conversion is correct, run.

In [ ]:
!python src/convert_annotations.py --dataset flickr27 --plot

# Split dataset
Next, we need to partition the dataset into train, validation, and test sets. These will contain 80%, 10%, and 10% of the data, respectively.

In [ ]:
!python src/prepare_data.py --dataset flickr27

# Download base models

In [ ]:
if YOLO_VERSION == 'yolov7':
    # Download YOLOv7 models
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt -P src/
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt -P src/
else:
    # Download YOLOv11 models using the provided script
    !sh src/download_models.sh

# Train the model

In [ ]:
if YOLO_VERSION == 'yolov7':
    # YOLOv7 Training
    !python src/yolov7/train.py \
        --img-size 640 \
        --cfg src/cfg/training/yolov7.yaml \
        --hyp data/hyp.scratch.yaml \
        --batch 2 \
        --epoch 300 \
        --data data/logo_data_flickr.yaml \
        --weights src/yolov7_training.pt \
        --workers 2 \
        --name yolo_logo_det \
        --device 0
else:
    # YOLOv11 Training
    # First, import ultralytics
    from ultralytics import YOLO
    
    # Initialize the model
    model = YOLO('yolo11n.pt')
    
    # Define training configuration
    config = {
        'data': 'data/logo_data_flickr.yaml',
        'epochs': 300,
        'imgsz': 640,
        'batch': 16,  # Adjust based on your GPU memory
        'name': 'yolo11_logo_det',
        'device': '0',  # Auto-select device
        'workers': 8
    }
    
    # Train the model
    results = model.train(**config)

# Evaluate the model

In [ ]:
if YOLO_VERSION == 'yolov7':
    # YOLOv7 Validation
    !python src/yolov7/test.py \
        --img-size 640 \
        --batch 16 \
        --data data/logo_data_flickr.yaml \
        --weights runs/train/yolo_logo_det/weights/best.pt \
        --task test \
        --name yolo_logo_det_val
else:
    # YOLOv11 Validation
    metrics = model.val()

# Run inference

In [ ]:
if YOLO_VERSION == 'yolov7':
    # YOLOv7 Inference
    !python src/yolov7/detect.py \
        --source data/Sample/test \
        --weights runs/train/yolo_logo_det/weights/best.pt \
        --conf 0.25 \
        --name yolo_logo_det_inference
else:
    # YOLOv11 Inference
    model_path = 'runs/detect/yolo11_logo_det/weights/best.pt'
    inference_model = YOLO(model_path)
    
    # Run inference
    results = inference_model.predict(
        source='data/Sample/test',
        conf=0.25,
        save=True,
        name='yolo11_logo_det_inference'
    )

# Display results (YOLOv11 only)

In [ ]:
if YOLO_VERSION == 'yolo11':
    import matplotlib.pyplot as plt
    from PIL import Image
    import os
    
    # Get a list of result images
    result_dir = 'runs/detect/predict'
    result_images = [os.path.join(result_dir, img) for img in os.listdir(result_dir) if img.endswith(('.jpg', '.png'))]
    
    # Display a few results
    plt.figure(figsize=(20, 10))
    for i, img_path in enumerate(result_images[:5]):
        if i >= 5: break  # Show max 5 images
        plt.subplot(1, 5, i+1)
        img = Image.open(img_path)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Result {i+1}')
    plt.tight_layout()
    plt.show()